In [ ]:
!pip install huggingface_hub==0.26.1 -Uqq -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install -Uqq sagemaker -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
!pip install -Uqq packaging==24.1 -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
!pip install modelscope -i https://pypi.tuna.tsinghua.edu.cn/simple -Uqq

In [ ]:
from modelscope.hub.snapshot_download import snapshot_download
from pathlib import Path

local_model_path = Path("./bge-m3-model")

local_model_path.mkdir(exist_ok=True)
model_name = "Xorbits/bge-m3"
commit_hash = "v0.0.1"

snapshot_download(model_name, revision=commit_hash, cache_dir=local_model_path)

In [ ]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [ ]:
s3_model_prefix = "GenBI-Model/bge-m3-model"  # folder where model checkpoint will go
if region in ['cn-north-1', 'cn-northwest-1']:
    model_snapshot_path = f'{local_model_path}/{model_name}'
else:
    model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "GenBI-Model/bge_m3_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

In [ ]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

In [ ]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118"
    
)

#中国区需要替换为下面的image_uri
if region in ['cn-north-1', 'cn-northwest-1']:
    inference_image_uri = (
        f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.23.0-deepspeed0.9.5-cu118"
    )

print(f"Image going to be used is ---- > {inference_image_uri}")

In [ ]:
!mkdir -p bge_m3_deploy_code


In [ ]:
%%writefile bge_m3_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os
# from FlagEmbedding import FlagModel
from FlagEmbedding import BGEM3FlagModel

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'--device={device}')

def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")

    # model =  FlagModel(model_location)
    model = BGEM3FlagModel(model_location, use_fp16=True)
    
    return model

model = None

def handle(inputs: Input):
    global model
    if not model:
        model = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = None
    inputs = data["inputs"]
    if isinstance(inputs, list):
        input_sentences = inputs
    else:
        input_sentences =  [inputs]
        
    is_query = None
    if "is_query" in data:
        is_query = data["is_query"]
    instruction = None
    if "instruction" in data:  
        instruction = data["instruction"]
    max_length = 8192
    if max_length in data:
        max_length = data["max_length"]
    logging.info(f"inputs: {input_sentences}")
    logging.info(f"is_query: {is_query}")
    logging.info(f"instruction: {instruction}")
    
    if is_query and instruction:
        input_sentences = [ instruction + sent for sent in input_sentences ]
        
    sentence_embeddings =  model.encode(input_sentences, max_length=max_length)['dense_vecs']
    
    #result = {"sentence_embeddings": sentence_embeddings}
    return Output().add_as_json(sentence_embeddings)

In [ ]:
print(f"option.s3url ==> s3://{bucket}/{s3_model_prefix}/")


In [ ]:
import os

if not os.path.exists("bge_m3_deploy_code"):
    os.mkdir("bge_m3_deploy_code")

with open('bge_m3_deploy_code/serving.properties', 'w') as f:
    f.write("engine=Python")
    f.write("\n")
    f.write("option.tensor_parallel_degree=1")
    f.write("\n")
    f.write(f"option.s3url=s3://{bucket}/{s3_model_prefix}/")

In [ ]:
%%writefile bge_m3_deploy_code/requirements.txt
-i https://pypi.tuna.tsinghua.edu.cn/simple
FlagEmbedding==1.2.3
transformers==4.42.0
torch>=2.0.0

In [ ]:
!rm s2e_model.tar.gz
!cd bge_m3_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf s2e_model.tar.gz bge_m3_deploy_code

In [ ]:
s3_code_artifact = sess.upload_data("s2e_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

In [ ]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base("bge-m3") #Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

In [ ]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

In [ ]:
def get_vector_by_sm_endpoint(questions, sm_client, endpoint_name):
    parameters = {
    }

    response_model = sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(
            {
                "inputs": questions,
                "is_query": True,
                "instruction" :  "Represent this sentence for searching relevant passages:"
            }
        ),
        ContentType="application/json",
    )
    # 中文instruction => 为这个句子生成表示以用于检索相关文章：
    json_str = response_model['Body'].read().decode('utf8')
    json_obj = json.loads(json_str)
    embeddings = json_obj
    return embeddings

In [ ]:
prompts1 = ["你好啊，大聪明"]

emb = get_vector_by_sm_endpoint(prompts1, smr_client, endpoint_name)
print(emb)